In [18]:
import pandas as pd
import requests
import emsi
import queries
from sqlalchemy import create_engine
from secrets import secrets

### Get access to Emsi

In [19]:
access_token = emsi.get_access_token(secrets['emsi_client_id'], secrets['emsi_secret'])
emsi.check_health(access_token)

{'healthy': True, 'message': 'Service is healthy'}

### Load existing skills

In [3]:
f = open("skills/initial.txt", "r")
skills = []
for x in f:
    skills.append(x[:-1])

len(skills)

225

In [4]:
df_own = pd.DataFrame(columns=['Skill'])
df_own['Skill'] = skills
df_own.head()

,Skill
0,AB Testing
1,AB testing
2,AI
3,AIML
4,API


### Query existing skills

In [5]:
training_engine = create_engine(secrets['training_db'])
df_train = pd.read_sql_query(queries.skills_query, con=training_engine)
training_engine.dispose()
df_train.head()

,Name
0,Linear Algebra
1,Basic Calculus
2,DBMS
3,Hadoop
4,R Programming


### Get related skills from Emsi

In [6]:
def get_related_skills(skills, print_progress=True):
    new_skills = []
    for i, s in enumerate(skills):
        if (i + 1) % int(len(skills) / 10) == 0:
            print("{}% done".format((i + 1) / len(skills) * 100))
        related = emsi.get_emsi_skills(s, access_token)
        for r in related:
            new_skills.append(r['name'])
    return new_skills

In [7]:
# From own
related_own = get_related_skills(df_own['Skill'].to_list())
len(related_own)

9.777777777777779% done
19.555555555555557% done
29.333333333333332% done
39.111111111111114% done
48.888888888888886% done
58.666666666666664% done
68.44444444444444% done
78.22222222222223% done
88.0% done
97.77777777777777% done


842

In [8]:
df1 = pd.DataFrame(columns=['Skill'])
df1['Skill'] = related_own
df1.head()

,Skill,Source
0,Computer-Aided Design,Own
1,Air Brakes,Own
2,Aircraft Maintenance,Own
3,IBM AIX,Own
4,Basic First Aid,Own


In [9]:
# From own
related_training = get_related_skills(df_train['Name'].to_list())
len(related_training)

8.823529411764707% done
17.647058823529413% done
26.47058823529412% done
35.294117647058826% done
44.11764705882353% done
52.94117647058824% done
61.76470588235294% done
70.58823529411765% done
79.41176470588235% done
88.23529411764706% done
97.05882352941177% done


134

In [10]:
df2 = pd.DataFrame(columns=['Skill'])
df2['Skill'] = related_training
df2.head()

,Skill,Source
0,Linear Algebra,Training
1,Elementary Matrix (Linear Algebra),Training
2,Basic Linear Algebra Subprograms,Training
3,Numerical Linear Algebra,Training
4,Linear Algebra Package (LAPACK),Training


### Consolidate

In [12]:
df_emsi = pd.concat([df1, df2])
df_emsi.head()

,Skill,Source
0,Computer-Aided Design,Own
1,Air Brakes,Own
2,Aircraft Maintenance,Own
3,IBM AIX,Own
4,Basic First Aid,Own


In [13]:
df_emsi.describe(include='object')

,Skill,Source
count,976,976
unique,745,2
top,Recurrent Neural Network (RNN),Own
freq,6,842


In [14]:
df_emsi = df_emsi.drop_duplicates(subset=['Skill'])
df_emsi = df_emsi.sort_values(by=['Skill'])
df_emsi = df_emsi.reset_index().drop(columns=['index'])
df_emsi.head()

,Skill,Source
0,.NET Framework,Own
1,3D Modeling,Own
2,API Design,Own
3,API Gateway,Own
4,API Management,Own


In [15]:
df_emsi.describe(include='object')

,Skill,Source
count,745,745
unique,745,2
top,Test Execution Engine,Own
freq,1,722


In [16]:
df_emsi.to_csv('skills/emsi_indeed.csv')

### Extract skills from text

In [12]:
def extract_skills(text_list, print_progress=True):
    new_skills = []
    for i, text in enumerate(text_list):
        if (i + 1) % int(len(text_list) / 10) == 0:
            print("{}% done".format((i + 1) / len(text_list) * 100))
        skills = emsi.extract_skills(text, access_token)
        for s in skills:
            new_skills.append(s['name'])
    return new_skills

In [6]:
df_indeed = pd.read_csv('resource/indeed_data_scientist.csv')
df_indeed.head()

,Unnamed: 0,Job_ID,Job_Title,Company,Salary,Country,State,Location,Metadata,Date_Posted,Description,Job_URL,Keywords_Present,Title_Keywords
0,0,21ceb0b509803172,Data Scientist,Chứng khoán VNDIRECT,NaN,Vietnam,Not Applicable,Hà Nội,NaN,2020-12-03,Trở về trang trướcChia sẻData ScientistNơi Làm...,https://vn.indeed.com/rc/clk?jk=21ceb0b5098031...,"'C', 'R', 'SQL', 'Statistics', 'Tableau', 'Pow...",NaN
1,0,33cda4aa961449fb,"Associate Scientist, Regulatory Affairs",Procter & Gamble,NaN,Thailand,Not Applicable,NaN,NaN,NaN,Are you passionate about making sure every det...,https://th.indeed.com/rc/clk?jk=33cda4aa961449...,"'C', 'Artificial Intelligence', 'R'","'Associate', 'Assoc'"
2,0,bdc6afc932fd4d07,Data Engineer,Chứng khoán VNDIRECT,NaN,Vietnam,Not Applicable,Hà Nội,NaN,2020-12-03,Trở về trang trướcChia sẻData EngineerNơi Làm ...,https://vn.indeed.com/rc/clk?jk=bdc6afc932fd4d...,"'Kafka', 'Airflow', 'Spark', 'C', 'R', 'SQL', ...",NaN
3,0,2407740df8f3c799,Data Scientist,Swivelt Sdn. Bhd.,"RM 4,000 - RM 5,000 a month",Malaysia,Not Applicable,Kuala Lumpur,NaN,2020-12-02,The Data Scientist department is constantly lo...,https://www.indeed.com.my/company/Swivelt-Sdn....,"'SAS', 'Tableau', 'SQL', 'Excel', 'NoSQL', 'Py...",NaN
4,0,5198579f85df40f6,"Data Scientist, Visa Consulting & Analytics (C...",Visa,NaN,Thailand,Not Applicable,กรุงเทพมหานคร,NaN,2020-12-01,Company DescriptionAs the world's leader in di...,https://th.indeed.com/rc/clk?jk=5198579f85df40...,"'Regression', 'Factor', 'SAS', 'SQL', 'Excel',...",NaN


In [10]:
df_50jobs = df_indeed.loc[df_indeed['Country'] == 'Singapore']
df_50jobs = df_50jobs.drop_duplicates(subset=['Description'])
df_50jobs = df_50jobs.loc[df_50jobs['Description'] != 'No Description']
df_50jobs = df_50jobs.sample(n=50)
df_50jobs.describe(include='object')

,Job_ID,Job_Title,Company,Salary,Country,State,Location,Date_Posted,Description,Job_URL,Keywords_Present,Title_Keywords
count,50,50,50,22,50,50,29,29,50,50,50,12
unique,50,44,45,16,1,1,6,5,50,50,41,5
top,fd244435b906dd0f,Data Scientist,Reeracoen Singapore Pte Ltd,"$1,000 - $1,500 a month",Singapore,Not Applicable,Singapore,2020-12-04,Applied Materials is an innovation-driven comp...,https://www.indeed.com.sg/rc/clk?jk=3701fadca9...,"'C', 'R'",'Senior'
freq,1,5,2,2,50,50,20,13,1,1,5,8


In [46]:
new_skills = []
url = "https://emsiservices.com/skills/versions/latest/extract"

for job in df_50jobs['Description'].to_list():
    payload = {'text': job, 'confidenceThreshold': 0.4}
    headers = {'authorization': 'Bearer {}'.format(access_token), 'content-type': 'text/plain'}
    response = requests.request("POST", url, data=payload, headers=headers)
    print(response.text)
    skills = response.json()['data']
    for s in skills:
        new_skills.append(s['name'])

{"attributions": [{"name": "Wikipedia", "text": "Wikipedia extracts are distributed under the CC BY-SA license (https://creativecommons.org/licenses/by-sa/3.0/)"}], "data": []}
{"attributions": [{"name": "Wikipedia", "text": "Wikipedia extracts are distributed under the CC BY-SA license (https://creativecommons.org/licenses/by-sa/3.0/)"}], "data": []}
{"attributions": [{"name": "Wikipedia", "text": "Wikipedia extracts are distributed under the CC BY-SA license (https://creativecommons.org/licenses/by-sa/3.0/)"}], "data": []}
{"attributions": [{"name": "Wikipedia", "text": "Wikipedia extracts are distributed under the CC BY-SA license (https://creativecommons.org/licenses/by-sa/3.0/)"}], "data": []}
{"attributions": [{"name": "Wikipedia", "text": "Wikipedia extracts are distributed under the CC BY-SA license (https://creativecommons.org/licenses/by-sa/3.0/)"}], "data": []}
{"attributions": [{"name": "Wikipedia", "text": "Wikipedia extracts are distributed under the CC BY-SA license (htt

KeyError: 'data'